In [ ]:
### QC
import json

nshot = 0
for task in ['1','2','3','4','5','5_qa_split','5_mc_split','0_knowledge','0_mc_CMExam','0_lt_split']:

    print('-'*25,task,'-'*25)
    for model in ['psychfound']:
        if task == '5': nshot=0
        output_pth = './result/API/{}shot/task{}_{}.json'.format(nshot,task,model)
        lines = []
        id = 1
        total = 0
        failed = []
        try:
            with open(output_pth, 'r', encoding="utf-8") as f:
                for line in f:
                    total += 1
                    data = json.loads(line)
                    answer_key = [key for key in data.keys() if 'answer_' in key][0]
                    if data[answer_key] == 'API调用失败' or data[answer_key] == 'failed':
                        failed.append(id)
                    id += 1
            print(model.ljust(20),'\t',total,'\t',len(failed),'\t',failed)
        except:
            print(output_pth,'not exist')


In [ ]:
### gathering ans for task5 qa and mc

import os
import json 


def form_ans(ans_list):
    ans = ''
    for i in range(len(ans_list)):
        ans += f'[答案{i+1}]'+ans_list[i].strip('\n') + '\n'
    return ans



root_path = './result/API/'


for shot in ['0']:
    for task in ['task5_mc_split','task5_qa_split']:
            
            for model in ['psychfound']:
                ans_fail_cnt = 0
                file_path = os.path.join(root_path,'{}shot/{}_{}.json'.format(shot,task,model))
                answers = []
                # question_dict = {}
                answer_dict = {}
                reasoning_dict = {}
                with open(file_path,'r',encoding='utf-8') as f:
                    for line in f.readlines():
                        answers.append(json.loads(line))
                print(len(answers))
                for ans in answers:
                    # print(ans['id'])
                    reasoning_ans = ans['answer_0']
                    reasoning = reasoning_ans.split('</think>')[0]
                    if '</think>' in reasoning_ans:
                        ans['answer_0'] = reasoning_ans.split('</think>')[-1].replace('<answer>','').replace('</answer>','')
                    elif '<answer>' in reasoning_ans:
                        ans['answer_0'] = reasoning_ans.split('<answer>')[-1]
                    else:
                        # print(reasoning_ans)
                        ans_fail_cnt += 1
                        ans['answer_0'] = reasoning_ans
                        

                    if task == 'task5_qa_split':
                        ans['id'] = ans['id'].split('-')[0]+'-'+ str(int(ans['id'].split('-')[-1])//3)
                    if ans['id'] in answer_dict.keys():
                        answer_dict[ans['id']].append(ans['answer_0'])
                        reasoning_dict[ans['id']].append(reasoning)
                    else:
                         answer_dict[ans['id']] = [ans['answer_0']]
                         reasoning_dict[ans['id']]=[reasoning]
                        #  question_dict[ans['id']] = [ans['query'].split('[问题]')[-1]]

                if task == 'task5_qa_split':
                    ref_path = os.path.join(root_path,'{}shot/{}_{}.json'.format(shot,'task5_qa','gpt-4'))
                if task == 'task5_mc_split':
                    ref_path = os.path.join(root_path,'{}shot/{}_{}.json'.format(shot,'task5_mc','gpt-4'))
                
                ref_answers = []
                with open(ref_path,'r',encoding='utf-8') as f:
                     for line in f.readlines():
                        ref_answers.append(json.loads(line))
                print(len(ref_answers))

                for i in range(len(ref_answers)):
                    ref_answers[i]['answer_0'] = form_ans(answer_dict[ref_answers[i]['id']])
                    ref_answers[i]['reasoning_0'] = reasoning_dict[ref_answers[i]['id']]
                    
                # print(ref_answers[0])
                    # print(ans)
                    # print(answer_dict)
                
                if task == 'task5_qa_split':
                    output_pth = os.path.join(root_path,'{}shot/{}_{}.json'.format(shot,'task5_qa',model))
                if task == 'task5_mc_split':
                    output_pth = os.path.join(root_path,'{}shot/{}_{}.json'.format(shot,'task5_mc',model))

                writer = open(output_pth, "w", encoding='utf-8')
                for line in ref_answers:
                    writer.write(json.dumps(line, ensure_ascii=False) + "\n")

                print('output to', output_pth)
                print('failed ans cnt', ans_fail_cnt)

In [ ]:
### reformat json
import json
import os
nshot = 0

os.makedirs('./result-refined/API/0shot',exist_ok=True)

for task in ['1','2','3','4','5','5_qa','5_mc','0_mc_CMExam','0_lt_split','0_knowledge']:

    for model in ['psychfound']:
        if task == '5' or task == '5_exam': nshot = 0
        output_pth = './result/API/{}shot/task{}_{}.json'.format(nshot,task,model)
        lines = []
        try:
            with open(output_pth, 'r', encoding="utf-8") as f:
                for line in f:
                    lines.append(json.loads(line))
        except:
            print(output_pth,'not exist')
            continue
        with open(output_pth.replace('result','result-refined'), 'w', encoding="utf-8") as f:
            json.dump(lines, f, ensure_ascii=False, indent=4)
        output_pth = output_pth.replace('result','result-refined')
        print(f'output to {output_pth}')

In [ ]:
# split reasoning and ans content
import os
import json

for task in ['1','2','3','4','0_knowledge','0_lt_split','0_mc_CMExam']:

    print('='*50,task)
    for model in ['psychfound']:
        new_data = []
        with open('./result-refined/API/0shot/task{}_{}.json'.format(task,model),'r', encoding='utf-8') as f:    
            data = json.load(f)
            cut_cnt = 0
            for adata in data:
                # print(adata['answer_0'])
                if '<think>' in adata['answer_0'] or '</think>' in adata['answer_0']:
                    if '</think>' in adata['answer_0']:
                        # reasoning = adata['answer_0'].split('<think>')[-1].split('</think>')[0]
                        reasoning = '</think>'.join(adata['answer_0'].split('</think>')[:-1])
                        answer = adata['answer_0'].split('</think>')[-1]
                    else:
                        if '<answer>' in adata['answer_0']:
                            reasoning = '<answer>'.join(adata['answer_0'].split('<answer>')[:-1])
                            answer = adata['answer_0'].split('<answer>')[-1]
                        else:
                            reasoning = adata['answer_0']
                            answer = '<think>'.join(adata['answer_0'].split('<think>')[:-1])
                            # print(reasoning)
                            cut_cnt += 1
                            print(adata['id'])

                    # if adata['id'] == "Anding-clinical-21":
                    #     a = 1
                    if len(answer.replace('\n','').replace('<answer>','').replace('</answer>','').replace(' ','')) < 10 and task in ['1','2','3','4']:
                        answer = reasoning + answer
                    # answer = answer.replace('<answer>','').replace('</answer>','')

                    answer = answer.replace('<think>','').replace('</think>','').replace('<answer>','').replace('</answer>','')
                    while '\n\n' in answer:
                        answer = answer.replace('\n\n','\n').strip('\n')

                    adata['answer_0'] = answer
                    adata['reasoning_0'] = reasoning
                else:
                    adata=adata
                new_data.append(adata)
                # print(adata['answer_0'])
                # break
            f.close()
        with open('./result-refined/API/0shot/task{}_{}.json'.format(task,model),'w', encoding='utf-8') as f:
            json.dump(new_data, f, ensure_ascii=False, indent=4)
        print('reasoning failed cnt:',cut_cnt)

In [ ]:
# check num of refined ans

nshot = 0

for task in ['1','2','3','4','5_qa','5_mc','0_lt_split','0_knowledge','0_mc_CMExam']:
    print('-'*25,task,'-'*25)

    for model in ['psychfound']:
    
        failed = []
        if task == '5_qa' or task == '5_mc' or task == '0_lt_split' or task == '0_knowledge' or task == '0_mc_CMExam': nshot = 0
        
        output_pth = '{}/{}shot/task{}_{}.json'.format('result-refined/API',nshot,task,model)
        lines = []
        try:
            with open(output_pth, "r", encoding="utf-8") as f:
                sub_answers = json.load(f)
            for ans in sub_answers:
                answer_key = [key for key in ans.keys() if 'answer_' in key][0]
                id = ans['id']
                if ans[answer_key] == 'API调用失败':
                    failed.append(id)
            print(model.ljust(30),'\t',len(sub_answers),'\t',len(failed),'\t',failed)
        except:
            print(output_pth,'not exist')
            continue

### Calc NER Score

In [ ]:
### calc ner 
### First: Set kernal to tf1!
# requirements
# Keras==2.2.4
# matplotlib==3.4.0
# pandas==1.2.3
# tensorflow==1.14.0
# tqdm==4.61.2

import os
nshot = 0
for task in [1,3,'5_qa']:
    print('-'*50,'task{}'.format(task),'-'*50)
    for model in ['psychfound']:  
        print(model)
        if task == 5:
            nshot=0

        ans_path = './result-refined/API/{}shot/task{}_{}.json'.format(nshot,task,model)
        dir_out = './src/ner_result/PsychClinical/{}shot/{}/task{}'.format(nshot,model,task)


        if not os.path.exists(ans_path):
            print('Error:',ans_path,'not exist!!!')
            continue

        ## 若想要生成cot格式的数据，加上--use_cot \ 目前只有Exam的选择题可用
        os.system('conda activate tf1')
        os.system('python -u ./chinese_medical_ner/ccksyidu4k-ner-roformer/evaluate_ner.py \
            --ans_path={} \
            --dir_out={} \
            > output_calc_ner.log 2>&1'.format(ans_path,dir_out))

In [ ]:
## check ner result
import numpy as np
data = np.load('./src/ner_result/PsychClinical/0shot/psychfound/task1/ner_cls_result.npy',allow_pickle=True)
data1 = np.load('./src/ner_result/PsychClinical/0shot/psychfound/task1/ner_result.npy',allow_pickle=True)
print(len(data))
print(data[0])
print(data1[0])

In [ ]:
### calc MNER-Precision Recall F1-score

import numpy as np

model = 'psychfound'
ner_path = './src/ner_result/PsychClinical/0shot/{}/task3/ner_result.npy'.format(model)
ner_result = np.load(ner_path,allow_pickle=True)
# print(ner_result)


p_list,r_list,f_list = [],[],[]

for result in ner_result:
    tgt_list,out_list = result

    # tgt_list = set(tgt_list)
    # out_list = set(out_list)
    
    intersection_precision = 0
    intersection_recall = 0

    for tgt in tgt_list:
        for out in out_list:
            if (tgt in out) or (out in tgt):
                intersection_recall += 1
    
    for out in out_list:
        for tgt in tgt_list:
            if (tgt in out) or (out in tgt):
                intersection_precision += 1

    intersection = set(tgt_list).intersection(set(out_list))


    if not out_list or not tgt_list:
        continue

    # precision
    precision = intersection_precision / len(out_list) if out_list else 0

    # recall
    recall = intersection_recall / len(tgt_list) if tgt_list else 0

    # F1 score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0


    p_list.append(precision)
    r_list.append(recall)
    f_list.append(f1_score)

p_list = np.array(p_list)
r_list = np.array(r_list)
f_list = np.array(f_list)

print(np.mean(p_list),',',np.std(p_list))
print(np.mean(r_list),',',np.std(r_list))
print(np.mean(f_list),',',np.std(f_list))

In [ ]:
# ### calc MNER-BERTScore
# ### First: Set kernal to py310

# ## calc with cpu
# ## try
# import time
# from transformers import BertTokenizer, BertModel
# import torch
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity

# # load model
# tokenizer = BertTokenizer.from_pretrained("F:/LM/model_zoo/bert-base-chinese") ## "bert-base-chinese"
# bert_model = BertModel.from_pretrained("F:/LM/model_zoo/bert-base-chinese")


# tgt_list = ['乏力感', '厌世', '躯体不适', '社会功能严重受损', '兴趣减退', '言行紊乱', '脑器质性疾病', '情绪低落', '精神障碍', '情绪差伴躯体不适', '焦虑', '自责']
# out_list = ['认知行为治疗', '与家人交流障碍', '偶有轻生想法', '长期适应性障碍', '利培酮', '心理治疗', '发呆', '独处时感到被支配', '沉迷学佛后出现精神异常', '与家人交流困难', '急性而短暂的精神病性障碍', '兴趣减退', '有被害妄想和攻击行为', '言语紊乱', '自知力可', '攻击行为', '心脏', '肝脏', '反应慢', '持续的情绪低落', '沉迷学佛', '氢溴酸西酞普兰', '无法胜任家务', '情绪低落', '急性起病', '被害妄想', '社会功能受损', '重度抑郁发作']
# print(len(tgt_list))
# print(len(out_list))

# def get_word_embedding(word):
   
#     input_ids = tokenizer.encode(word, add_special_tokens=True, return_tensors='pt')
    
#     with torch.no_grad():
#         output = bert_model(input_ids)
    
#     return output.last_hidden_state[:, 0, :].numpy()


# def calculate_similarity_matrix(words_list1, words_list2):
#     if len(words_list1) > 0 and len(words_list2) > 0:
#         embeddings1 = np.array([get_word_embedding(word) for word in words_list1])
#         embeddings2 = np.array([get_word_embedding(word) for word in words_list2])
        
#         # 计算余弦相似性矩阵
#         similarity_matrix = cosine_similarity(embeddings1.reshape(embeddings1.shape[0],-1), embeddings2.reshape(embeddings2.shape[0],-1))
#     else:
#         similarity_matrix = np.zeros((2,2))
#     return similarity_matrix

# time1 = time.time()
# similarity_matrix = calculate_similarity_matrix(tgt_list, out_list)
# time2 = time.time()
# time_cost = time2-time1
# print(time_cost)

# print(similarity_matrix)
# print(np.max(np.array(similarity_matrix),axis=1).mean())


In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import time
from transformers import BertTokenizer, BertModel,AlbertTokenizer,AlbertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# https://huggingface.co/iioSnail/bert-base-chinese-medical-ner
tokenizer = BertTokenizer.from_pretrained("iioSnail/bert-base-chinese-medical-ner")
bert_model = BertModel.from_pretrained("iioSnail/bert-base-chinese-medical-ner")


# load model to gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)


tgt_list = ['乏力感', '厌世', '躯体不适', '社会功能严重受损', '兴趣减退', '言行紊乱', '脑器质性疾病', '情绪低落', '精神障碍', '情绪差伴躯体不适', '焦虑', '自责']
out_list = ['认知行为治疗', '与家人交流障碍', '偶有轻生想法', '长期适应性障碍', '利培酮', '心理治疗', '发呆', '独处时感到被支配', '沉迷学佛后出现精神异常', '与家人交流困难', '急性而短暂的精神病性障碍', '兴趣减退', '有被害妄想和攻击行为', '言语紊乱', '自知力可', '攻击行为', '心脏', '肝脏', '反应慢', '持续的情绪低落', '沉迷学佛', '氢溴酸西酞普兰', '无法胜任家务', '情绪低落', '急性起病', '被害妄想', '社会功能受损', '重度抑郁发作']
print(len(tgt_list))
print(len(out_list))

# 批量获取词向量
def get_word_embeddings(words, batch_size=32):
    all_embeddings = []
    for i in range(0, len(words), batch_size):
        batch = words[i: i + batch_size]
        # 对单词进行编码
        input_ids = tokenizer(batch, add_special_tokens=True, return_tensors='pt', padding=True, truncation=True)
        input_ids = {k: v.to(device) for k, v in input_ids.items()}
        # 获取词向量
        with torch.no_grad():
            output = bert_model(**input_ids)
        # 使用[CLS]标记的向量作为句子向量
        embeddings = output.last_hidden_state[:, 0, :].cpu().numpy()
        all_embeddings.extend(embeddings)
        # 及时释放显存
        if device.type == 'cuda':
            torch.cuda.empty_cache()
    return np.array(all_embeddings)

# 计算相似性矩阵
def calculate_similarity_matrix(words_list1, words_list2):
    if len(words_list1) > 0 and len(words_list2) > 0:
        embeddings1 = get_word_embeddings(words_list1)
        embeddings2 = get_word_embeddings(words_list2)
        
        # 计算余弦相似性矩阵
        similarity_matrix = cosine_similarity(embeddings1, embeddings2)
    else:
        similarity_matrix = np.zeros((2,2))
    return similarity_matrix

time1 = time.time()
# 计算tgt和out列表的相似性矩阵
similarity_matrix = calculate_similarity_matrix(tgt_list, out_list)
time2 = time.time()
time_cost = time2 - time1
print(time_cost)

# 打印相似性矩阵
print(similarity_matrix)
print(np.max(np.array(similarity_matrix), axis=1).mean())

In [ ]:
### calc MNER-BERTScore

import os
import shutil

nshot = 0
for task in [1,3,'5_qa']:

    print('-'*50,'task{}'.format(task),'-'*50)
    for model in ['psychfound']:
    
        print(model)
        if task == 5:
            nshot=0

        ans_path = './result-refined/API/{}shot/task{}_{}.json'.format(nshot,task,model)
        dir_out = './src/ner_result/PsychClinical/{}shot/{}/task{}'.format(nshot,model,task)

        ner_path = dir_out +'/ner_result.npy'

        if not os.path.exists(ner_path):
            print('Error:',ner_path,'not exist!!!')
            continue
        ner_result = np.load(ner_path,allow_pickle=True)
        ner_scores = []
        for tgt,out in ner_result:
            similarity_matrix = calculate_similarity_matrix(list(set(tgt)), list(set(out)))
            
            # print(similarity_matrix)
            ner_score = np.max(np.array(similarity_matrix),axis=0).mean()
            if ner_score == 0:
                continue
            ner_scores.append(ner_score)
        ner_scores_mean = np.mean(ner_scores)
        ner_scores_std = np.std(ner_scores)
        print('ner score:{}±{}'.format(ner_scores_mean,ner_scores_std))
        
        ### save
        import json

        # Load the uploaded JSON file
        file_path = dir_out +'/metrics.json'

        file_path_new = dir_out +'/metrics_new.json'
        shutil.copyfile(file_path,file_path_new)

        # Read the content of the file
        with open(file_path_new, 'r') as file:
            metrics = json.load(file)
        
        np.save(dir_out + '/MNER-BERTScore.npy',ner_scores)

        # Display the content of the JSON file to understand its structure
        metrics['NER-score'] = {'avg':float(ner_scores_mean),'std':float(ner_scores_std)}
        with open(file_path_new, 'w') as file:
            file.write(json.dumps(metrics))
            

### Calc other score

In [ ]:
### calc BLEU ROUGE BERTScore
# pip install evaluate
NSHOT = 0
import os

nshot = NSHOT
for task in [1,3,'5_qa']:
    print('-'*50,'task{}'.format(task),'-'*50)

    for model in ['psychfound']:    

        print(model)

        ans_path = './result-refined/API/{}shot/task{}_{}.json'.format(nshot,task,model)
        dir_out = './result/PsychClinical/{}shot/{}_api/task{}'.format(nshot,model,task)

        if not os.path.exists(ans_path):
            print('Error:',ans_path,'not exist!!!')
            continue

        os.system('python -u ./src/calc_metrics.py \
            --ans_path={} \
            --dir_out={} \
            > output_calc.log 2>&1'.format(ans_path,dir_out))

In [ ]:
## calc Acc etc.

### calc metrics
import tqdm
import numpy as np 
import os
import json 

NSHOT = 0

## integrity
def compute_integrity(ans):
    cnt = 0
    if '病程标准' in ans or '病程' in ans: cnt += 1
    if '症状学标准' in ans or '症状标准' in ans: cnt += 1
    if '严重程度标准' in ans or '严重标准': cnt += 1
    if '排除标准' in ans or '排除诊断' in ans: cnt += 1
    # if cnt/4*100 != 100:
    #     print(ans)
    return cnt/4*100

import re
def find_time(text):
    # 使用正则表达式匹配时间描述
    time_patterns = [
        r'\d+年',
        r'\d+月',
        r'\d+天',
        # r'\d+余',
        r'\d+余日',
        r'\d+余月',
        r'\d+余年'
    ]

    times_found = []
    for pattern in time_patterns:
        times_found.extend(re.findall(pattern, text))

    # 去重并排序结果
    unique_times = sorted(set(times_found))
    return unique_times

def find_des(text):
    # 使用正则表达式匹配时间描述
    des_patterns = [
        r'慢性',
        r'急性',
        r'持续',
        r'连续',
        r'间断',
        r'间歇',
        r'亚急性',
        r'反复',
        r'波动',
    ]

    des_found = []
    for pattern in des_patterns:
        des_found.extend(re.findall(pattern, text))

    # 去重并排序结果
    unique_times = sorted(set(des_found))
    if '急性' in unique_times and '亚急性' in unique_times:
        unique_times = [a for a in unique_times if a != '急性']
    
    return unique_times

import re
def find_icd(text):
    # text = "患者的主要诊断是 F30.901，需要进行进一步的治疗。"
    icd_dict = {}
    with open('./data/psych/icd-10.jsonl','r',encoding='utf-8') as f:
        for line in f.readlines():
            data = json.loads(line)
            icd_dict[data["ICD-10 Name"]] = data['ICD-10 Code']
    icd10_codes = re.findall(r'\bF\d+\.*\d+\b', text)
    if len(icd10_codes) < 1:
        find_name = ''
        for name in icd_dict.keys():
            if name in text and len(name) > len(find_name):
                find_name = name
        if find_name == '':
            print('failed to find icd in:',text)
        return ''
    return icd10_codes[0]  # 输出: ['F30.901']

def acc_task1(out,tgt):
    try:
        out_zs = out.split('主诉')[1].split('病程标准')[0].split('病例特点')[0].replace('：','').replace(':','').replace('\n','')
        out_zs = re.split('诊断标准|病程标准|症状学标准|症状标准|严重程度标准|严重标准|排除标准|排除诊断|病例特点',out_zs)[0]
    except:
        out_zs = out.split('病程标准')[0]
        # print(out)
    tgt_zs = tgt.split('主诉：')[1].split('病例特点')[0].replace('\n','')
    tgt_zs = re.split('诊断标准|病程标准|症状学标准|症状标准|严重程度标准|严重标准|排除标准|排除诊断|病例特点',tgt_zs)[0]
    # print(out_zs,'|',tgt_zs)

    hit = 0
    tgt_num = 0
    
    times_zs = find_time(tgt_zs)
    tgt_num += len(times_zs)
    for t in times_zs:
        if t in out_zs:
            hit += 1

    if '病程标准' in out:
        out_bc = out.split('病程标准')[1].split('症状学标准')[0].replace('：','').replace(':','').replace('\n','').replace('*','')
        out_bc = re.split(r'\d+\.',re.split('症状学标准|症状标准|严重程度标准|严重标准|排除标准|排除诊断|病例特点',out_bc)[0])[0]
    else:
        try:
            out_bc = re.split('病例特点|诊断标准|病程标准',out)[1]
        except:
            out_bc = ''
    try:
        tgt_bc = tgt.split('病程标准：')[1].split('2.')[0].replace('\n','')
    except:
        tgt_bc = tgt.split('排除诊断')[0].split('病例特点')[1]
    key_bc =  find_time(tgt_bc) + find_des(tgt_bc)
    tgt_num += len(key_bc)
    for k in key_bc:
        if k in out_bc:
            hit += 1
    
    if '严重程度标准' in out:
        out_yz = re.split('严重程度标准|严重标准',out)[1].split('排除标准')[0].replace('：','').replace(':','').replace('\n','').replace('*','')
    else:
        try:
            out_yz = re.split('病例特点|诊断标准|病程标准',out)[1]
        except:
            out_yz = ''
    try:
        tgt_yz = re.split('严重程度标准：|严重标准',tgt)[1].split('4.')[0].replace('\n','')
    except:
        tgt_yz = tgt.split('排除诊断')[0].split('病例特点')[1]
    opt = ['严重','明显','轻微']
    key_yz =  [o for o in opt if o in tgt_yz]
    tgt_num += len(key_yz)
    for k in key_yz:
        if k in out_yz:
            hit += 1

    try:
        acc = hit / tgt_num
    except:
        acc = 0
        print(tgt)
    return acc*100



def acc_task2(out,tgt):
    out_icd = find_icd(out.split('精神科共病诊断：')[0])
    icd_dict = {}
    with open('./data/psych/icd-10.jsonl','r',encoding='utf-8') as f:
        for line in f.readlines():
            data = json.loads(line)
            icd_dict[data["ICD-10 Name"]] = data['ICD-10 Code']
    try:
        tgt_name = re.split('精神科共病诊断：|鉴别诊断：',re.split('主要诊断：|明确诊断：',tgt)[1])[0].strip(' ').replace('\n','').replace('。','').replace('，',',')
        tgt_name = tgt_name.split(' ')[0]
        tgt_icd = icd_dict[tgt_name]
    except Exception as e:
        print(e)
        # print(tgt)
    
    # print(out_icd,tgt_icd)
    if len(out_icd) == 0:
        # print(out)
        acc = 0
        return 0
    acc = 0
    if '.' in tgt_icd:
        if out_icd[:3] == tgt_icd[:3]:
            acc = 0.5
        if out_icd == tgt_icd:
            acc = 1
    elif '-' in tgt_icd:
        if int(out_icd[1:3]) > int(tgt_icd.split('-')[0][1:3]) and int(out_icd[1:3]) < int(tgt_icd.split('-')[1][1:3]):
            acc = 1
    else:
        if int(out_icd[1:3]) == int(tgt_icd[1:3]):
            acc = 1
    # print(tgt_icd,'|',out_icd)
    return acc*100

### 修改为broad形式



def refine_name(text):
    names = ['人格障碍',
            '双相情感障碍',
            '器质性精神障碍',
            '复发性抑郁障碍',
            '妄想性障碍',
            '广泛性焦虑障碍',
            '强迫性障碍',
            '心境障碍',
            '急性而短暂的精神病性障碍',
            '抑郁发作',
            '抑郁障碍',
            '焦虑障碍',
            '环性心境',
            '精神分裂症',
            '精神障碍',
            '脑器质性精神病',
            '躁狂发作',
            '躯体症状障碍',
            '酒精所致的精神行为障碍',
            '阿尔兹海默',
            '阿尔茨海默']
    for name in names:
        if name in text:
            return name
    if len(re.findall(r'\“(.*?)\”',text)) > 0:
        return re.findall(r'\“(.*?)\”',text)[0]
    return text

def refine_name_new(text):
    text = text.replace('，',',')
    names = []
    with open('./data/psych/icd-10-broad.jsonl','r',encoding='utf-8') as f:
        for line in f.readlines():
            data = json.loads(line)
            names.append(data["ICD-10 Name"])
    refined_name = ''
    for name in names:
        if name in text:
            if len(name) > len(refined_name):
                refined_name = name
    if len(refined_name) > 0:        
        return refined_name
    if len(re.findall(r'\“(.*?)\”',text)) > 0:
        return re.findall(r'\“(.*?)\”',text)[0]
    return text


def acc_task3_1(out,tgt):
    
    out = out.replace('：',':').replace(',','，')
    try:
        out_list = re.findall(r'\[(.*?)\]', out)
        out_list = [out for out in out_list if out!= '疾病名称']
        out_main = out_list[0]
        out_dif1 = out_list[1]
        out_dif2 = out_list[2]

    except:
        try:
            out_main = out.split('主要诊断')[1].split('\n')[0].replace('[','').replace(']','').replace(':','')
            out_dif1 = out.split('<鉴别诊断1>')[1].split('\n')[0].split('，')[0].replace('[','').replace(']','').replace(':','')
            out_dif2 = out.split('<鉴别诊断2>')[1].split('\n')[0].split('，')[0].replace('[','').replace(']','').replace(':','')
        except:
            try:
                out_main_part = out.split('鉴别诊断:')[0]
                out_main = refine_name(out_main_part)
                if out_main == out_main_part:
                    out_main = ''
            except:
                out_main = ''
            try:
                out_dif1 = out.split('<鉴别诊断1>')[1].split('<鉴别诊断2>')[0].split('鉴别点:')[0].replace('，','')
            except:
                
                try:
                    out_dif1 = out.split('鉴别诊断:')[1].split(':')[0].split('，')[0].replace('，','')
                except:
                    try:
                        if '-' in out.split('鉴别诊断')[-1]:
                            out_dif1 = out.split('鉴别诊断')[-1].split('-')
                            if len(out_dif1) >= 3:
                                out_dif1 = out_dif1[-2].split(':')[0].split('，')[0].replace('，','')
                            else:
                                out_dif1 = out_dif1[1].split(':')[0].split('，')[0].replace('，','')
                        elif '1' in out.split('鉴别诊断')[-1]:
                            out_dif1 = out.split('鉴别诊断')[-1].split('1')[-1].split(':')[0].split('，')[0].replace('，','')
                        else:
                            out_dif1 = out.split('鉴别诊断')[-1].split('，')[0].split(':')[-1]
                    except:
                        print(out)
                        out_dif1 = ''
            try:
                out_dif2 = out.split('<鉴别诊断2>')[1].split('鉴别点:')[0].replace('，','')
            except:
                # print(out)
                try:
                    out_dif2 = out.split('鉴别诊断:')[1].split('2')[-1].split('-')[-1].split(':')[0].split('，')[0].replace('，','')
                except:
                    try:
                        if '-' in out.split('鉴别诊断')[-1]:
                            out_dif2 = out.split('鉴别诊断')[-1].split('-')
                            if len(out_dif1) >= 3:
                                out_dif2 = out_dif2[-1].split(':')[0].split('，')[0].replace('，','')
                            else:
                                out_dif2 = ''
                        elif '2' in out.split('鉴别诊断')[-1]:
                            out_dif2 = out.split('鉴别诊断')[-1].split('2')[-1].split(':')[0].split('，')[0].replace('，','')
                        else:
                            out_dif2 = out.split('鉴别诊断')[-1].replace('\n\n','\n').split('\n')[-1].split('，')[0].split(':')[-1]
                    except:
                        print(out)
                        out_dif2 = ''

    out_main = refine_name(out_main)
    out_dif1 = refine_name(out_dif1)
    out_dif2 = refine_name(out_dif2) 
    if fuzzy_match(out_main,tgt['main']):
        acc_main = 1
    else:
        acc_main = 0
    
    acc_diff = 0
    if fuzzy_match_list(out_dif1,tgt['diff']):
        acc_diff += 0.5
    if fuzzy_match_list(out_dif2,tgt['diff']):
        acc_diff += 0.5
    print(out_main,'|',out_dif1,'|',out_dif2)
    return acc_main*100,acc_diff*100


def find_icd_broad(text):
    # text = "患者的主要诊断是 F30.901，需要进行进一步的治疗。"
    icd_dict = {}
    with open('./data/psych/icd-10-broad.jsonl','r',encoding='utf-8') as f:
        for line in f.readlines():
            data = json.loads(line)
            icd_dict[data["ICD-10 Name"]] = data['ICD-10 Code']
    icd10_codes = re.findall(r'\bF\d+\.*\d+\b', text)
    refined_name = ''
    if len(icd10_codes) < 1:
        for name in icd_dict.keys():
            if name in text:
                if len(name) > len(refined_name):
                    refined_name = name
        if len(refined_name) > 0:
            return icd_dict[refined_name]
        # print('failed to find icd in:',text)
        return ''
    return icd10_codes[0]  # 输出: ['F30.901']

def calc_icd_acc(out_icd,tgt_icd):
    acc= 0
    if len(out_icd) > 0 and len(tgt_icd) > 0:
        if '-' not in out_icd and '-' not in tgt_icd:
            if out_icd[1:3] == tgt_icd[1:3]:
                acc = 1
        elif '-' in out_icd and '-' not in tgt_icd:
            if int(tgt_icd[1:3]) >= int(out_icd.split('-')[0][1:3]) and int(tgt_icd[1:3]) <= int(out_icd.split('-')[1][1:3]):
                acc = 0.5
            else:
                acc = 0
        elif '-' not in out_icd and '-' in tgt_icd:
            if int(out_icd[1:3]) >= int(tgt_icd.split('-')[0][1:3]) and int(out_icd[1:3]) <= int(tgt_icd.split('-')[1][1:3]):
                acc = 1
            else:
                acc = 0
        elif '-' in out_icd and '-' in tgt_icd:
            if out_icd == tgt_icd:
                acc = 1
            else:
                acc = 0
        else:
            acc = 0
    elif len(out_icd) == 0:
        acc = 0
    else:
        acc = 1
    return acc



def acc_task3_2(out,tgt):
    out = out.replace('［','[').replace('］',']')
    out = out.replace('：',':').replace(',','，').replace('[转换]','（转换）').replace('[情感]','(情感)')
    try:
        out_list = re.findall(r'\[(.*?)\]', out)
        out_list = [out for out in out_list if '疾病名称' not in out]
        out_main = out_list[0]
        out_dif1 = out_list[1]
        out_dif2 = out_list[2]

    except:
        try:
            out_main = '主要诊断'.join(out.split('主要诊断')[1:]).split('鉴别诊断')[0].replace('[','').replace(']','').replace(':','')
            out_dif1 = out.split('鉴别诊断1')[1].split('\n')[0].split('，')[0].replace('[','').replace(']','').replace(':','')
            out_dif2 = out.split('鉴别诊断2')[1].split('\n')[0].split('，')[0].replace('[','').replace(']','').replace(':','')
        except:
            try:
                # out_main_part = out.split('鉴别诊断:')[0]
                out_main_part = re.split(r'鉴别诊断[0-9]?:', out)[0]
                if '诊断:' in out_main_part:
                    out_main_part = ''.join(out_main_part.split('诊断:')[1:])
                out_main = refine_name_new(out_main_part)
                if out_main == out_main_part:
                    out_main = ''
            except:
                out_main = ''
            try:
                out_dif1 = out.split('<鉴别诊断1>')[1].split('<鉴别诊断2>')[0].split('鉴别点:')[0].replace('，','')
            except:
                
                try:
                    out_dif1 = out.split('鉴别诊断:')[1].split(':')[0].split('，')[0].replace('，','')
                except:
                    try:
                        if '-' in out.split('鉴别诊断')[-1]:
                            out_dif1 = out.split('鉴别诊断')[-1].split('-')
                            if len(out_dif1) >= 3:
                                out_dif1 = out_dif1[-2].split(':')[0].split('，')[0].replace('，','')
                            else:
                                out_dif1 = out_dif1[1].split(':')[0].split('，')[0].replace('，','')
                        elif '1' in out.split('鉴别诊断')[-1]:
                            out_dif1 = out.split('鉴别诊断')[-1].split('1')[-1].split(':')[0].split('，')[0].replace('，','')
                        else:
                            out_dif1 = out.split('鉴别诊断')[-1].split('，')[0].split(':')[-1]
                    except:
                        print(out)
                        out_dif1 = ''
            try:
                out_dif2 = out.split('<鉴别诊断2>')[1].split('鉴别点:')[0].replace('，','')
            except:
                # print(out)
                try:
                    out_dif2 = out.split('鉴别诊断:')[1].split('2')[-1].split('-')[-1].split(':')[0].split('，')[0].replace('，','')
                except:
                    try:
                        if '-' in out.split('鉴别诊断')[-1]:
                            out_dif2 = out.split('鉴别诊断')[-1].split('-')
                            if len(out_dif1) >= 3:
                                out_dif2 = out_dif2[-1].split(':')[0].split('，')[0].replace('，','')
                            else:
                                out_dif2 = ''
                        elif '2' in out.split('鉴别诊断')[-1]:
                            out_dif2 = out.split('鉴别诊断')[-1].split('2')[-1].split(':')[0].split('，')[0].replace('，','')
                        else:
                            out_dif2 = out.split('鉴别诊断')[-1].replace('\n\n','\n').split('\n')[-1].split('，')[0].split(':')[-1]
                    except:
                        print(out)
                        out_dif2 = ''

    out_main = refine_name_new(out_main)
    out_dif1 = refine_name_new(out_dif1)
    out_dif2 = refine_name_new(out_dif2) 
    
    ### 模糊匹配
    if fuzzy_match(out_main,tgt['main']):
        acc_main = 1
    else:
        acc_main = 0
    
    acc_diff = 0
    if fuzzy_match_list(out_dif1,tgt['diff']):
        acc_diff += 0.5
    if fuzzy_match_list(out_dif2,tgt['diff']):
        acc_diff += 0.5
    # print(out_main,'|',out_dif1,'|',out_dif2)

    ### ICD
    out_main_icd = find_icd_broad(out_main)
    out_dif1_icd = find_icd_broad(out_dif1)
    out_dif2_icd = find_icd_broad(out_dif2)

    tgt_main_icd = find_icd_broad(tgt['main'])
    tgt_diffs_icd = [find_icd_broad(tgt_diff) for tgt_diff in tgt['diff']]

    acc_main = calc_icd_acc(out_main_icd,tgt_main_icd)
    
    tgt_diffs_icd = [icd for icd in tgt_diffs_icd if len(icd)>0]
    acc_diff = 0
    if len(tgt_diffs_icd) > 1:
        acc_diff += max(calc_icd_acc(out_dif1_icd,tgt_diffs_icd[0]),calc_icd_acc(out_dif1_icd,tgt_diffs_icd[1]))
        acc_diff += max(calc_icd_acc(out_dif2_icd,tgt_diffs_icd[0]),calc_icd_acc(out_dif2_icd,tgt_diffs_icd[1]))
        acc_diff = acc_diff/2
    elif len(tgt_diffs_icd) == 1:
        acc_diff += calc_icd_acc(out_dif1_icd,tgt_diffs_icd[0])
        acc_diff += calc_icd_acc(out_dif2_icd,tgt_diffs_icd[0])
    else:
        print('couldn\'t find tgt diff:',tgt['diff'])
        acc_diff = 1
    # print(out_main_icd,'|',out_dif1_icd,'|',out_dif2_icd)
    # print(tgt_main_icd,'|',tgt_diffs_icd)
    # print('='*20)

    return acc_main*100,acc_diff*100


from fuzzywuzzy import fuzz

def fuzzy_match(str1, str2):
    """
    使用 Fuzzywuzzy 库进行模糊匹配，返回两个字符串之间的相似度百分比。
    """
    ratio = fuzz.ratio(str1, str2)
    
    return ratio
    
def fuzzy_match_list(str1, tgt_list):
    """
    使用 Fuzzywuzzy 库进行模糊匹配，返回两个字符串之间的相似度百分比。
    """
    flag = False
    ratio = 0
    for str2 in tgt_list:
        tmp_ratio = fuzzy_match(str1,str2)
        if tmp_ratio > ratio:
            ratio = tmp_ratio
            # print(str2,ratio)
    return ratio

def hit_task4(out,tgt):
    # drugs = ['丙戊酸', '伏硫西汀', '利培酮', '喹硫平', '奥氮平', '安非他酮', '帕利哌酮', '帕罗西汀', '度洛西汀', '拉莫三嗪', '文拉法辛', '氟伏沙明', '氟西汀', '米氮平', '舍曲林', '艾司西酞普兰', '西酞普兰', '阿戈美拉汀', '阿立哌唑', '马普替林', '齐拉西酮']
    drugs = ['丙戊酸','伏硫西汀','利培酮', '安非他酮','劳拉西泮','唑吡坦','喹硫平','地西泮','奋乃静','奥氮平','奥沙西泮','帕利哌酮','帕罗西汀','度洛西汀','拉莫三嗪','文拉法辛','曲唑酮','氟伏沙明','氟哌啶醇','氟西汀','氨磺必利','氯丙嗪','氯氮平','硝西泮','碳酸锂','米氮平','美金刚','舍曲林','艾司西酞普兰','西酞普兰','阿戈美拉汀','阿立哌唑','鲁拉西酮','齐拉西酮']
    out = out.replace('，',',').replace(' ','').replace('\n\n','\n').replace('：',':')
    tgt = tgt.replace('，',',').replace(' ','')
    try:
        if '推荐药物' in out:
            out_chunks = '推荐药物'.join(out.split('推荐药物')[1:])
        else:
            out_chunks = out.split('\n')
            if len(out_chunks) == 1:
                out_chunks = out_chunks
            elif '分析' in out_chunks[0]:
                out_chunks = out_chunks[1:]
            # out_chunks = out.split('\n')[1:]
        
        if isinstance(out_chunks,str):
            out_chunks = [out_chunks]
        ans_idx = 0
        max_match = 0
        for idx,piece in enumerate(out_chunks):
            match = 0
            for drug in drugs:
                if drug in piece: match += 1
            if match>max_match:
                max_match = match
                ans_idx = idx

        # out_list = out_chunks[ans_idx].split(':')[-1].replace('。','') #.split(',')#.split('、')
        out_list = []
        for drug in drugs:
            if drug in out_chunks[ans_idx]:
                out_list.append(drug)
        # 按照在答案中出现的顺序排序
        out_list.sort(key=lambda x: out_chunks[ans_idx].find(x))
    except:
    #     out_list = out.split('分析')[0].split(',').split(' ')
    
    # if len(out_list[0]) > 6:
        out_list = []
        for drug in drugs:
            if drug in '\n'.join(out.split('推荐药物')[1:]):
                out_list.append(drug)
        out_list.sort(key=lambda x: ('\n'.join(out.split('推荐药物')[1:])).find(x))
    
    if len(out_list) ==0:
        out_list = []
        for drug in drugs:
            if drug in '\n'.join(out.split('\n')[1:]):
                out_list.append(drug)
        out_list.sort(key=lambda x: ('\n'.join(out.split('\n')[1:])).find(x))
        
    if len(out_list) ==0:  
        out_list = ['']
        # print('failed to match drug:',out)

    
    tgt_list = tgt.split(',')
    
    
    o_in_t = [d for d in out_list if d in tgt_list]
    t_in_o = [d for d in tgt_list if d in out_list]
    acc = 0
    if out_list[0]==tgt_list[0]:
        acc = 1
    recall = len(t_in_o) / len(tgt_list)
    precision = len(o_in_t) / len(out_list)
    
    return acc*100,precision*100,recall*100

def recall_task4(out,tgt):
    # drugs = ['丙戊酸', '伏硫西汀', '利培酮', '喹硫平', '奥氮平', '安非他酮', '帕利哌酮', '帕罗西汀', '度洛西汀', '拉莫三嗪', '文拉法辛', '氟伏沙明', '氟西汀', '米氮平', '舍曲林', '艾司西酞普兰', '西酞普兰', '阿戈美拉汀', '阿立哌唑', '马普替林', '齐拉西酮']
    drugs = ['丙戊酸','伏硫西汀','利培酮', '安非他酮','劳拉西泮','唑吡坦','喹硫平','地西泮','奋乃静','奥氮平','奥沙西泮','帕利哌酮','帕罗西汀','度洛西汀','拉莫三嗪','文拉法辛','曲唑酮','氟伏沙明','氟哌啶醇','氟西汀','氨磺必利','氯丙嗪','氯氮平','硝西泮','碳酸锂','米氮平','美金刚','舍曲林','艾司西酞普兰','西酞普兰','阿戈美拉汀','阿立哌唑','鲁拉西酮','齐拉西酮']
    out = out.replace('，',',').replace(' ','').replace('\n\n','\n').replace('：',':')
    tgt = tgt.replace('，',',').replace(' ','')
    
    tgt_list = tgt.split(',')
    hit_list = []
    for tgt in tgt_list:
        if tgt in out:
            hit_list.append(tgt)
    recall = len(hit_list) / len(tgt_list)
    
    return recall*100


def task5_exam_acc(out,tgt):
    correct = 0
    ans = re.findall(r'[A-E]\.', out)
    if len(ans)<len(tgt):
        ans = re.findall(r'[A-E]', out)
        if len(ans)<len(tgt):
            # print(out)
            # print('-'*50)
            if len(ans) < 1:
                return 0
            else:
                for a,t in zip(ans,tgt[:len(ans)]):
                    if a==t:
                        correct += 1
                return correct/len(tgt)*100
    else:
        ans = [a.replace('.','') for a in ans]

    ans = ans[-len(tgt):]
    
    for a,t in zip(ans,tgt):
        if a==t:
            correct += 1
    return correct/len(tgt)*100

def task5_exam_acc_2(out,tgt):
    tgt_options,tgt_contents = tgt
    correct = 0
    if '[' in out:
        ans_list1 = out.split('[')[1:]
    else:
        ans_list1 = []
    
    out = out.replace('\n\n','\n').strip(' ').strip('\n')
    ans_list2 = out.split('\n')

    if len(ans_list1) > len(ans_list2) or len(re.findall(r'\[', out)) == len(tgt_options):
        ans_list = ans_list1
    else:
        ans_list = ans_list2

    if len(ans_list) > len(tgt_options):
        ans_list = ans_list[-len(tgt_contents):]
    # if len(ans_list) != len(tgt_options):
    #     print(out)
    #     print('='*50)

    correct = 0
    for t_opt,t_con,a in zip(tgt_options,tgt_contents,ans_list):
        if len(re.findall(r'[A-Ea-e]', a)) > 0:
            if t_opt == re.findall(r'[A-Ea-e]', a)[0].upper():
                correct += 1
                continue
            if (t_con in a) or (a in t_con):
                correct += 1
                continue
    return correct/len(tgt_options)*100


def calc_ner_prf(ner_tgt,ner_out):
    # ner_tgt = list(set(ner_tgt))
    # ner_out = list(set(ner_out))

    # intersection = len(set(ner_out).intersection(set(ner_tgt)))
    # intersection_tgt = intersection
    # intersection_out = intersection

    intersection_tgt = 0
    intersection_out = 0

    for en in ner_tgt:
        for en_ in ner_out:
            if en in en_ or en_ in en or en==en_:
                intersection_tgt += 1
                break

    for en in ner_out:
        for en_ in ner_tgt:
            if en in en_ or en_ in en or en==en_:
                intersection_out += 1
                break

    # for en in ner_out:
    #     if fuzzy_match_list(en,ner_tgt) > 80:
    #         intersection += 1
    # ner_tgt = list(set(ner_tgt))
    # ner_out = list(set(ner_out))
    precision = intersection_out / len(ner_out) if ner_out else 0
    recall = intersection_tgt / len(ner_tgt) if ner_tgt else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision*100,recall*100,f1*100



### calc

nshot = NSHOT
metrics_list = []   
task3_ans = np.load('./data/psych/task3_ans.npy',allow_pickle=True).item()
  
for task in ['1','2','3','4','5_qa','5_mc']:
    print('-'*100)
    print('task',task)

    for model in ['psychfound']: 
        if task == '5_exam' or task =='5': nshot = 0
        ans_path = './result-refined/API/{}shot/task{}_{}.json'.format(nshot,task,model)
        if task in ['1','3','5_qa']:
            ner_path = './src/ner_result/PsychClinical/{}shot/{}/task{}/ner_result.npy'.format(nshot,model,task) #.replace('task3','task3-refined')
            ner_result = np.load(ner_path,allow_pickle=True)
            # print(ner_path,len(ner_result))
        if not os.path.exists(ans_path):
            print(ans_path,'not exist')
            continue
        # load data
        lst_tgt = []
        lst_out = []
        lst_idx = []
        lst_ipid = []
        lst_tgt_option,lst_tgt_content = [],[]
        option_qa = [[],[]]
        with open(ans_path, "r", encoding="utf-8") as f:
            answers = json.load(f)
        idx = 0
        for ans in answers:
            if 'question_type' not in ans.keys():
                ans['question_type'] = 'clinical'
            if '选择题' in ans['question_type']:
                option_qa[0].append(ans['answer'])
                option_qa[1].append(ans['model_answer'])
                idx += 1
            else:
                if ans['question_type'] == 'clinical':
                    if task == '5_mc':
                        lst_tgt_option.append(ans['answer'])
                        tgt_content = ans['conversations'][1]['value'].split('\n')
                        tgt_content = ['.'.join(content.split('.')[1:]) for content in tgt_content]
                        lst_tgt_content.append(tgt_content)
                        lst_tgt.append([ans['answer'],tgt_content])
                    elif ans['conversations'][1]['from']=='gpt':
                        lst_tgt.append(ans['conversations'][1]['value'])
                    else:
                        lst_tgt.append(ans['conversations'][2]['value'])
                else:
                    lst_tgt.append(ans['answer'])
                # lst_out.append(ans['model_answer'])
                lst_out.append(ans['answer_0'])
                lst_ipid.append(ans['id'])
                lst_idx.append(idx)
                idx += 1

        # compute scores of each sample across entire dataset
        scores_all = {}
        failed = 0
        # for tgt, out, idx in tqdm.tqdm(zip(lst_tgt, lst_out, lst_idx)):
        for tgt, out, idx, ipid in zip(lst_tgt, lst_out, lst_idx,lst_ipid):

            if out == 'API调用失败' or out == 'failed':
                failed += 1
                continue
            # get sub-dict containing scores for each metric
            # task1
            if 'task1' in ans_path:
                metrics = ['integrity','Acc']
                # scores = [compute_integrity(out),acc_task1(out,tgt)]
                try:
                    scores = [compute_integrity(out),acc_task1(out,tgt)]
                except Exception as e:
                    print('Error!',e)
                    print(ipid)
                    print(out,tgt)

            
            # task2
            if 'task2' in ans_path:
                metrics = ['Acc']
                scores = acc_task2(out,tgt)
                # try:
                #     scores = acc_task2(out,tgt)
                # except Exception as e:
                #     print(e)
                #     print(ipid)
                    # exit()
            
            # task3
            if 'task3' in ans_path:
                metrics = ['Acc_main','Acc_diff']
                tgt = task3_ans[ipid]
                # scores = acc_task3(out,tgt)
                scores = acc_task3_2(out,tgt)
                # try:
                #     scores = acc_task3_2(out,tgt)
                # except Exception as e:
                #     print(e)
                #     print(out)

            
            # task4
            if 'task4' in ans_path:
                metrics = ['Acc','precision','recall']
                # print('-'*50)
                # print(out)
                scores = list(hit_task4(out,tgt))
                # recall  = recall_task4(out,tgt)
                # scores[-1] = recall


            # task5 exam
            if 'task5_mc' in ans_path:
                metrics = ['Acc']
                # print('-'*50)
                # print(out)
                scores = max(task5_exam_acc(out,tgt[0]),task5_exam_acc_2(out,tgt))

            if 'task1' in ans_path or 'task3' in ans_path or 'task5' in ans_path:
                if 'task5_mc' in ans_path:
                    pass
                else:
                    if 'task5_qa' in ans_path:
                        metrics = []
                        scores = []
                    ner_tgt,ner_out = ner_result[idx-failed]
                    ner_scores = calc_ner_prf(ner_tgt,ner_out)
                    if 'NER-Precision' not in metrics:
                        metrics.extend(['NER-Precision','NER-Recall','NER-F1'])
                    # metrics = list(set(metrics))
                    scores = list(scores)
                    scores.extend(ner_scores)
                    # print(ner_scores)
            # append to master dict, dataset object
            scores_all[idx] = scores

        # print(metrics)
        print(model,np.mean(np.array(list(scores_all.values())),0),np.std(np.array(list(scores_all.values())),0))
        avgs = np.mean(np.array(list(scores_all.values())),0,keepdims=False)
        stds = np.std(np.array(list(scores_all.values())),0,keepdims=False)
        if len(metrics) > 1:
            for metric,avg,std in zip(metrics,avgs,stds):
                metrics_list.append({'Task':'task{}'.format(task.replace('_mc','')),'Model': model, 'Metric': metric, 'Average': avg, 'Standard Deviation': std})
        else:
            metrics_list.append({'Task':'task{}'.format(task.replace('_mc','')),'Model': model, 'Metric': metrics[0], 'Average': avgs, 'Standard Deviation': stds})

### save metrics

In [ ]:
import pandas as pd
import json
import os

import time

# 获取当前时间
current_time = time.localtime()

# 格式化日期和时间
formatted_date_time = time.strftime("%Y%m%d-%H%M", current_time)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def summarize_metrics_from_files(directory, file_pattern="metrics.json",metrics_list=None,nshot=None):
    # 初始化一个空的DataFrame来存储所有模型的指标
    if 'ner_result' in directory:
        file_pattern="metrics_new.json" # calc with https://huggingface.co/iioSnail/bert-base-chinese-medical-ner 
    all_metrics = pd.DataFrame(columns=['Task','Model','Metric', 'Average', 'Standard Deviation'])
    if metrics_list is None:
        metrics_list = []
    # 遍历指定目录
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file == file_pattern:
                # 构建完整的文件路径
                file_path = os.path.join(root, file)
                
                # 从文件中读取数据
                with open(file_path, 'r') as f:
                    data = json.load(f)
                

                task = os.path.basename(root)
                # if task == 'task5' or task == 'task3': continue
                if task == 'task5_qa': continue
                if task == 'task3-refined': continue

                # 提取模型名（假设文件路径的最后一部分是模型名）
                model_name = root.replace('\\','/').split('/')[-2].split('_')[0]
                
                # 提取指标并添加到DataFrame中
                for metric, scores in data.items():
                    if metric in ['NER-Precision','NER-Recall','NER-F1']: continue
                    if metric in ['NER-score']: 
                        avg = scores['avg'] * 100
                        std = scores['std'] * 100
                    else:
                        avg = scores['avg']
                        std = scores['std']
                    metrics_list.append({'Task':task.replace('-refined',''),'Model': model_name, 'Metric': metric, 'Average': avg, 'Standard Deviation': std})
                    # metrics_list.append({'Task':task,'Model': model_name, 'Metric': metric, 'Average': avg, 'Standard Deviation': std})

    
    for root, dirs, files in os.walk(directory.replace('1shot','0shot')):
        for file in files:
            if file == file_pattern:
                # 构建完整的文件路径
                file_path = os.path.join(root, file)
                
                # 从文件中读取数据
                with open(file_path, 'r') as f:
                    data = json.load(f)
                

                task = os.path.basename(root)
                if task != 'task5_qa': continue
                # 提取模型名（假设文件路径的最后一部分是模型名）
                model_name = root.replace('\\','/').split('/')[-2].split('_')[0]
                
                # 提取指标并添加到DataFrame中
                for metric, scores in data.items():
                    if metric in ['NER-Precision','NER-Recall','NER-F1']: continue
                    if metric in ['NER-score']: 
                        avg = scores['avg'] * 100
                        std = scores['std'] * 100
                    else:
                        avg = scores['avg']
                        std = scores['std']
                    metrics_list.append({'Task':task.replace('-refined',''),'Model': model_name, 'Metric': metric, 'Average': avg, 'Standard Deviation': std})
    
    all_metrics = pd.concat([pd.DataFrame(metrics_list)], ignore_index=True)

    # all_metrics.reset_index(drop=True, inplace=True)
    os.makedirs('./result_table/',exist_ok=True)
    all_metrics.to_csv('./result_table/summary_result_{}shot_{}.csv'.format(nshot,formatted_date_time), index=False)
    print('saving result to ','./result_table/summary_result_{}shot_{}.csv'.format(nshot,formatted_date_time))
    return all_metrics,metrics_list

NSHOT = 0
nshot = NSHOT
# metrics_list = None

# BLEU\ROUGE\BERTScore
directory_path = "./result/PsychClinical/{}shot".format(nshot)
summary_df,metrics_list = summarize_metrics_from_files(directory_path,metrics_list=metrics_list,nshot=nshot)
# print(summary_df)

# NER result
directory_path = "./src/ner_result/PsychClinical/{}shot".format(nshot)
summary_df,metrics_list = summarize_metrics_from_files(directory_path,metrics_list=metrics_list,nshot=nshot)
print(summary_df)

In [ ]:
import pandas as pd

# Load the uploaded files
table_df = pd.read_excel('./table_template.xlsx'.format(NSHOT))
summary_result_df = pd.read_csv('./result_table/summary_result_{}shot_{}.csv'.format(NSHOT,formatted_date_time))

# Display the first few rows of each dataframe to understand their structure
table_df.head(), summary_result_df.head()

In [ ]:
col_name_map = {
    'task1':{'BLEU':0,'ROUGE-L':1,'BERT':2,'integrity':3,'Acc':4,'NER-Precision':5,'NER-Recall':6,'NER-F1':7,'NER-score':8},
    'task2':{'Acc':0},
    'task3':{'BLEU':0,'ROUGE-L':1,'BERT':2,'Acc_main':3,'Acc_diff':4,'NER-Precision':5,'NER-Recall':6,'NER-F1':7,'NER-score':8},
    'task4':{'Acc':0,'recall':1,'precision':2},
    'task5':{'BLEU':0,'ROUGE-L':1,'BERT':2,'NER-Precision':3,'NER-Recall':4,'NER-F1':5,'NER-score':6,'Acc':7}
}



# Function to format the data as 'avg±std' and round to 2 decimal places
def format_data(avg, std):
    return f"{avg:.2f}±{std:.2f}"

# Update the matching logic to handle the differences in task names
def get_column_name(task, metric):
    # Handle special case for integrity metric
    name =  col_name_map[task][metric]
    if name == 0:
        name = task 
    else:
        name = f"{task}.{name}"
    # Handle other metrics by appending the metric name with a dot separator
    return name

# Iterate over each row in summary_result_df and update the corresponding cells in table_df
for index, row in summary_result_df.iterrows():
    model = row['Model']
    task = row['Task']  # Convert task1 to Task1 for matching
    metric = row['Metric']
    avg = row['Average']
    std = row['Standard Deviation']
    
    # Find the column name in table_df that matches the task and metric
    try:
        column_name = get_column_name(task, metric)
    except:
        continue
    if column_name in table_df.columns:
        # Find the row in table_df that matches the model
        model_row = table_df[table_df['model'] == model].index
        if not model_row.empty:
            # Update the cell with the formatted data
            # if column_name =='task1.8' or column_name =='task3.8' or column_name =='task5.6': 
                # avg = avg * 100
                # std = std * 100
            table_df.at[model_row[0], column_name] = format_data(avg, std)

table_df.to_excel('./result_table/table_new_{}shot_{}.xlsx'.format(NSHOT,formatted_date_time), index=False)

print('Saving result to ','./result_table/table_new_{}shot_{}.xlsx'.format(NSHOT,formatted_date_time))
# Display the updated table_df
table_df


### knowledge test

In [ ]:
### calc knowledge test

import tqdm
import numpy as np 
import os
import json 
import re
import pandas as pd

NSHOT = 0


def task0_lt_acc(out,tgt):
    tgt = [tgt]
    correct = 0
    ans = re.findall(r'[A-Ea-e]\.', out)
    # if len(ans) == 0:
    #     ans = re.findall(r'[A-Z]', out)
    if len(ans)<len(tgt):
        ans = re.findall(r'[A-Ea-e]', out)
        if len(ans)<len(tgt):
            # print(out)
            # print('-'*50)
            if len(ans) < 1:
                return 0
            else:
                for a,t in zip(ans,tgt[:len(ans)]):
                    if a==t:
                        correct += 1
                return correct/len(tgt)*100
    else:
        ans = [a.replace('.','') for a in ans]

    ans = ans[:len(tgt)]
    
    for a,t in zip(ans,tgt):
        if a.upper()==t.upper():
            correct += 1
    return correct/len(tgt)*100


def task0_lt_acc_2(out,tgt):
    tgt_option,tgt_content = tgt
    tgt_content = tgt_content.replace('，',',')
    out = out.replace('，',',')
    
    out = out.replace('\n\n','\n').strip(' ').strip('\n')
    

    ## match option 
    if len(re.findall(r'[A-Ea-e]', out)) > 0:
        if tgt_option == re.findall(r'[A-Ea-e]', out)[0].upper():
            return 100
    
    ## match content
    if (tgt_content in out) or (out in tgt_content):
        return 100
    
    return 0


### calc

nshot = NSHOT
metrics_list = []   
knowledge_detailed = pd.DataFrame(columns=['Task', 'Model', 'id', 'acc'])

def add_entry(task, model, ipid, scores, knowledge_detailed):
    if len(knowledge_detailed) == 0:
        knowledge_detailed = pd.DataFrame([{
            'Task': f'{task.replace("_exam", "")}',
            'Model': model,
            'id': ipid,
            'acc': scores
        }])
    else:
        existing_count = knowledge_detailed[(knowledge_detailed['Task']==task) & (knowledge_detailed['Model']==model)]['id'].str.startswith(ipid).sum()
        if existing_count > 0:
            new_ipid = f"{ipid}-{existing_count}"
        else:
            new_ipid = ipid
        new_row = pd.DataFrame([{
            'Task': f'{task.replace("_exam", "")}',
            'Model': model,
            'id': new_ipid,
            'acc': scores
        }])
        knowledge_detailed = pd.concat([knowledge_detailed, new_row], ignore_index=True)
    return knowledge_detailed


for task in ['0_mc_CMExam','0_knowledge','0_lt_split']:

    print('-'*100)
    print('task',task)
    for model in ['psychfound']: 

        if task == '0_lt_split' or task == '0_knowledge' or task == '0_mc_CMExam': nshot = 0
        ans_path = './result-refined/API/{}shot/task{}_{}.json'.format(nshot,task,model)
        
        if not os.path.exists(ans_path):
            print(ans_path,'not exist')
            continue
        # load data
        lst_tgt = []
        lst_out = []
        lst_idx = []
        lst_ipid = []
        lst_tgt_option,lst_tgt_content = [],[]
        option_qa = [[],[]]
        with open(ans_path, "r", encoding="utf-8") as f:
            answers = json.load(f)
        idx = 0
        for ans in answers:
            if 'question_type' not in ans.keys():
                ans['question_type'] = 'clinical'
            if '选择题' in ans['question_type']:
                option_qa[0].append(ans['answer'])
                option_qa[1].append(ans['model_answer'])
                idx += 1
            else:
                if ans['question_type'] == 'clinical':
                    if task == '0_lt_split' or task == '0_knowledge' or task=='0_mc_CMExam':
                        lst_tgt_option.append(ans['answer'])
                        tgt_content = '.'.join(ans['conversations'][1]['value'].split('.')[1:]) # drop A. B. etc in the beginning
                        lst_tgt_content.append(tgt_content)
                        lst_tgt.append([ans['answer'],tgt_content])
                    elif ans['conversations'][1]['from']=='gpt':
                        lst_tgt.append(ans['conversations'][1]['value'])
                    else:
                        lst_tgt.append(ans['conversations'][2]['value'])
                else:
                    lst_tgt.append(ans['answer'])
                # lst_out.append(ans['model_answer'])
                lst_out.append(ans['answer_0'])
                lst_ipid.append(ans['id'])
                lst_idx.append(idx)
                idx += 1

        # compute scores of each sample across entire dataset
        scores_all = {}
        failed = 0
        # for tgt, out, idx in tqdm.tqdm(zip(lst_tgt, lst_out, lst_idx)):
        for tgt, out, idx, ipid in zip(lst_tgt, lst_out, lst_idx,lst_ipid):

            if out == 'API调用失败' or out == None:
                failed += 1
                continue
            # get sub-dict containing scores for each metric
            if '0_lt_split' in ans_path or '0_knowledge' in ans_path or '0_mc_CMExam' in ans_path:
                metrics = ['Acc']
                # print('-'*50)
                # print(out)
                # try:
                #     scores = task0_lt_acc(out,tgt[0])
                # except:
                #     print(out)
                scores = task0_lt_acc_2(out,tgt)
                knowledge_detailed = add_entry('task{}'.format(task.replace('_exam','')),model,ipid,scores,knowledge_detailed)
            scores_all[idx] = scores
                
        print(model.ljust(30),np.mean(np.array(list(scores_all.values())),0))
        avgs = np.mean(np.array(list(scores_all.values())),0,keepdims=False)
        stds = np.std(np.array(list(scores_all.values())),0,keepdims=False)
        if len(metrics) > 1:
            for metric,avg,std in zip(metrics,avgs,stds):
                metrics_list.append({'Task':'task{}'.format(task.replace('_exam','')),'Model': model, 'Metric': metric, 'Average': avg, 'Standard Deviation': std})
        else:
            metrics_list.append({'Task':'task{}'.format(task.replace('_exam','')),'Model': model, 'Metric': metrics[0], 'Average': avgs, 'Standard Deviation': stds})

import pandas as pd
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(metrics_list)
df_detailed = pd.DataFrame(knowledge_detailed)
print(len(df_detailed))

# Save the DataFrame to an Excel file
file_path = './result_table/knowledge_test_result.xlsx'
df.to_excel(file_path, index=False)

file_path = './result_table/knowledge_test_result_detailed.xlsx'
df_detailed.to_excel(file_path, index=False)